In [1]:
!pip install geopandas pysal

print("--- 库安装完成 ---")

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.1/157.1 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 882.2/882.2 kB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.6/141.6 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.1/248.1 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.9/389.9 kB 28.0 MB/s eta 0:00:00
   ━━━━

In [3]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter # <--- 用于高效聚合计数
import glob  # 用于查找所有 parquet 文件
import libpysal.weights as lw # 用于空间权重
import esda # 用于空间统计
import os
from google.colab import drive

print("--- 库导入完成 ---")

# 挂载 Google Drive
print("--- 正在挂载 Google Drive ---")
try:
    drive.mount('/content/drive', force_remount=True)
    print("--- Google Drive 已成功挂载 ---")
except Exception as e:
    print(f"!!! 挂载 Google Drive 失败: {e} !!!")

--- 库导入完成 ---
--- 正在挂载 Google Drive ---
Mounted at /content/drive
--- Google Drive 已成功挂载 ---


In [5]:
# -----------------------------------------------
# 单元格 2: (脚本 A) 生成加权热点区域
# -----------------------------------------------
print("--- 开始执行脚本 A: 生成加权热点区域 ---")

# --- 1. 定义权重 (来自 temporal_weighting.md) ---
W_LUNCH = 1.0
W_WKDY_EVE = 1.2
W_WKND_EVE = 1.5
print(f"--- 权重已设置: 午餐={W_LUNCH}, 工作日晚餐={W_WKDY_EVE}, 周末晚餐={W_WKND_EVE} ---")

# --- 2. 加载 Taxi Zones ---
TAXI_ZONES_PATH = "/content/drive/MyDrive/Where-to-dine-demo/data/external/boundaries/taxi_zones.shp"
try:
    zones_gdf = gpd.read_file(TAXI_ZONES_PATH)
    if 'location_i' in zones_gdf.columns: # 处理列名不一致
        zones_gdf.rename(columns={'location_i': 'LocationID'}, inplace=True)
    zones_gdf['LocationID'] = zones_gdf['LocationID'].astype(int)
    print(f"--- 成功加载 {len(zones_gdf)} 个 Taxi Zones ---")
except Exception as e:
    print(f"!!! 致命错误: 加载 Taxi Zones 文件失败: {e} !!!")
    # 如果 taxi_zones 失败，则停止
    # raise e

# --- 3. 初始化三个计数器 ---
location_id_counts_lunch = Counter()
location_id_counts_wkdy_eve = Counter()
location_id_counts_wknd_eve = Counter()

# --- 4. 循环处理所有 Parquet 文件 ---
TAXI_DATA_PATH = "/content/drive/MyDrive/Where-to-dine-demo/data/raw/taxi/"
search_pattern = os.path.join(TAXI_DATA_PATH, '*.parquet')
parquet_files = glob.glob(search_pattern)

if not parquet_files:
    print(f"!!! 错误：在路径 {TAXI_DATA_PATH} 中未找到任何 '.parquet' 文件。")
else:
    print(f"--- 将处理 {len(parquet_files)} 个 Parquet 文件... ---")

    for file_num, file_path in enumerate(parquet_files, 1):
        print(f"  > 正在处理文件 {file_num}/{len(parquet_files)}: {os.path.basename(file_path)}")
        try:
            df = pd.read_parquet(file_path)

            # 重命名和检查列
            df.rename(columns={'DOLocationID': 'do_location_id', 'tpep_dropoff_datetime': 'dropoff_datetime'}, inplace=True, errors='ignore')
            if 'do_location_id' not in df.columns or 'dropoff_datetime' not in df.columns:
                print(f"    > 跳过: 缺少 'do_location_id' 或 'dropoff_datetime' 列。")
                continue

            # 转换日期并创建时间特征
            df['dropoff_datetime'] = pd.to_datetime(df['dropoff_datetime'])
            df['hour'] = df['dropoff_datetime'].dt.hour
            df['day_of_week'] = df['dropoff_datetime'].dt.dayofweek # 0=周一, 6=周日

            # --- 核心加权逻辑 (基于 temporal_weighting.md) ---
            is_lunch = (df['hour'] >= 11) & (df['hour'] <= 14)
            is_wkdy_eve = (df['day_of_week'] < 5) & (df['hour'] >= 17) & (df['hour'] <= 21)
            is_wknd_eve = (df['day_of_week'] >= 5) & (df['hour'] >= 17) & (df['hour'] <= 21)

            # 分别计数
            lunch_counts = df[is_lunch]['do_location_id'].value_counts()
            wkdy_eve_counts = df[is_wkdy_eve]['do_location_id'].value_counts()
            wknd_eve_counts = df[is_wknd_eve]['do_location_id'].value_counts()

            # 更新全局计数器
            location_id_counts_lunch.update(lunch_counts.to_dict())
            location_id_counts_wkdy_eve.update(wkdy_eve_counts.to_dict())
            location_id_counts_wknd_eve.update(wknd_eve_counts.to_dict())

        except Exception as e:
            print(f"    > 处理文件 {file_path} 时出错: {e}")

    print("--- 所有 Parquet 文件处理完毕 ---")

    # --- 5. 聚合和计算加权分数 ---
    df_lunch = pd.DataFrame.from_dict(location_id_counts_lunch, orient='index', columns=['Count_Lunch'])
    df_wkdy_eve = pd.DataFrame.from_dict(location_id_counts_wkdy_eve, orient='index', columns=['Count_WkdyEve'])
    df_wknd_eve = pd.DataFrame.from_dict(location_id_counts_wknd_eve, orient='index', columns=['Count_WkndEve'])

    # 合并所有 DataFrame
    df_counts = df_lunch.join(df_wkdy_eve, how='outer').join(df_wknd_eve, how='outer')
    df_counts.index.name = 'LocationID'
    df_counts.index = df_counts.index.astype(int)

    # 合并到地理文件
    gdf_weighted = zones_gdf.merge(df_counts, on='LocationID', how='left')

    # 填充 NaNs 为 0
    count_cols = ['Count_Lunch', 'Count_WkdyEve', 'Count_WkndEve']
    gdf_weighted[count_cols] = gdf_weighted[count_cols].fillna(0)

    # *** 计算加权分数 ***
    gdf_weighted['weighted_score'] = (
        (gdf_weighted['Count_Lunch'] * W_LUNCH) +
        (gdf_weighted['Count_WkdyEve'] * W_WKDY_EVE) +
        (gdf_weighted['Count_WkndEve'] * W_WKND_EVE)
    )

    print("--- 加权分数计算完成 ---")

    # --- 6. 运行 Pysal (在 'weighted_score' 上) ---
    print("--- 正在 'weighted_score' 上运行 Pysal (Getis-Ord Gi*)... ---")
    try:
        W = lw.Queen.from_dataframe(gdf_weighted)
        y = gdf_weighted['weighted_score']
        g_local = esda.G_Local(y, W)

        gdf_weighted['Zs'] = g_local.Zs
        gdf_weighted['p_sim'] = g_local.p_sim

        # 过滤热点
        hotspots_gdf = gdf_weighted[
            (gdf_weighted['Zs'] > 1.96) & (gdf_weighted['p_sim'] < 0.05)
        ]

        OUTPUT_FILE_A = "hotspot_arrival_areas_weighted.geojson"
        hotspots_gdf.to_file(OUTPUT_FILE_A, driver="GeoJSON")

        print(f"\n--- 成功！脚本 A 已完成 ---")
        print(f"识别出 {len(hotspots_gdf)} 个加权热点区域。")
        print(f"结果已保存到: {OUTPUT_FILE_A}")

    except Exception as e:
        print(f"\n!!! Pysal 分析失败: {e} !!!")

--- 开始执行脚本 A: 生成加权热点区域 ---
--- 权重已设置: 午餐=1.0, 工作日晚餐=1.2, 周末晚餐=1.5 ---
--- 成功加载 263 个 Taxi Zones ---
--- 将处理 12 个 Parquet 文件... ---
  > 正在处理文件 1/12: yellow_tripdata_2024-07.parquet
  > 正在处理文件 2/12: yellow_tripdata_2024-08.parquet
  > 正在处理文件 3/12: yellow_tripdata_2024-09.parquet
  > 正在处理文件 4/12: yellow_tripdata_2024-10.parquet
  > 正在处理文件 5/12: yellow_tripdata_2024-11.parquet
  > 正在处理文件 6/12: yellow_tripdata_2024-12.parquet
  > 正在处理文件 7/12: yellow_tripdata_2024-01.parquet
  > 正在处理文件 8/12: yellow_tripdata_2024-02.parquet
  > 正在处理文件 9/12: yellow_tripdata_2024-03.parquet
  > 正在处理文件 10/12: yellow_tripdata_2024-04.parquet
  > 正在处理文件 11/12: yellow_tripdata_2024-05.parquet
  > 正在处理文件 12/12: yellow_tripdata_2024-06.parquet
--- 所有 Parquet 文件处理完毕 ---
--- 加权分数计算完成 ---
--- 正在 'weighted_score' 上运行 Pysal (Getis-Ord Gi*)... ---


/tmp/ipython-input-3584546359.py:105: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  W = lw.Queen.from_dataframe(gdf_weighted)
/usr/local/lib/python3.12/dist-packages/libpysal/weights/contiguity.py:347: UserWarning: The weights matrix is not fully connected: 
 There are 10 disconnected components.
 There are 6 islands with ids: 0, 45, 102, 103, 104, 201.
  W.__init__(self, neighbors, ids=ids, **kw)
/usr/local/lib/python3.12/dist-packages/esda/getisord.py:527: RuntimeWarning: invalid value encountered in divide
  z_scores = (statistic - expected_value) / np.sqrt(expected_variance)


('WARNING: ', 0, ' is an island (no neighbors)')
('WARNING: ', 45, ' is an island (no neighbors)')
('WARNING: ', 102, ' is an island (no neighbors)')
('WARNING: ', 103, ' is an island (no neighbors)')
('WARNING: ', 104, ' is an island (no neighbors)')
('WARNING: ', 201, ' is an island (no neighbors)')

--- 成功！脚本 A 已完成 ---
识别出 23 个加权热点区域。
结果已保存到: hotspot_arrival_areas_weighted.geojson


/usr/local/lib/python3.12/dist-packages/esda/getisord.py:450: RuntimeWarning: invalid value encountered in divide
  self.z_sim = (self.Gs - self.EG_sim) / self.seG_sim


In [8]:
# -----------------------------------------------
# 单元格 3: (脚本 B & C) 生成最终多边形和餐厅数据库
# -----------------------------------------------

# --- (脚本 B) 识别 C: 最终热点餐饮区 ---
print("\n--- 开始执行脚本 B: 生成最终热点多边形 ---")

FILE_DINING_ZONES = "dining_zones.geojson"
FILE_HOTSPOT_ARRIVALS_WEIGHTED = "hotspot_arrival_areas_weighted.geojson"
OUTPUT_FILE_B = "final_hotspot_polygons_weighted.geojson"

try:
    # 加载两个输入文件
    gdf_dining_zones = gpd.read_file(FILE_DINING_ZONES)
    gdf_weighted_hotspots = gpd.read_file(FILE_HOTSPOT_ARRIVALS_WEIGHTED)

    # 确保 CRS 一致
    gdf_dining_zones = gdf_dining_zones.to_crs("EPSG:4326")
    gdf_weighted_hotspots = gdf_weighted_hotspots.to_crs("EPSG:4326")

    # 执行空间相交
    final_polygons_gdf = gpd.overlay(gdf_dining_zones, gdf_weighted_hotspots, how='intersection')
    final_polygons_gdf = final_polygons_gdf[~final_polygons_gdf.geometry.is_empty]

    final_polygons_gdf.to_file(OUTPUT_FILE_B, driver="GeoJSON")
    print(f"--- 成功！脚本 B 已完成 ---")
    print(f"识别出 {len(final_polygons_gdf)} 个最终融合的热点多边形。")
    print(f"结果已保存到: {OUTPUT_FILE_B}")

except FileNotFoundError as e:
    print(f"\n!!! 致命错误: 找不到文件 {e.filename} !!!")
    print("请确保 'dining_zones.geojson' 和 'hotspot_arrival_areas_weighted.geojson' 存在。")
except Exception as e:
    print(f"\n!!! 脚本 B 失败: {e} !!!")


# --- (脚本 C) 生成餐厅数据库 ---
print("\n--- 开始执行脚本 C: 生成餐厅数据库 ---")

FILE_RESTAURANTS_CSV = "restaurants_nyc_googlemaps.csv"
FILE_FINAL_POLYGONS_WEIGHTED = "final_hotspot_polygons_weighted.geojson"
OUTPUT_FILE_C = "restaurants_with_hotspot_scores.geojson"

try:
    # 加载餐厅 CSV
    df_restaurants = pd.read_csv(FILE_RESTAURANTS_CSV)
    df_restaurants = df_restaurants.dropna(subset=['latitude', 'longitude'])

    # 转换为 GeoDataFrame
    gdf_restaurants = gpd.GeoDataFrame(
        df_restaurants,
        geometry=gpd.points_from_xy(df_restaurants.longitude, df_restaurants.latitude),
        crs="EPSG:4326"
    )
    print(f"--- 成功加载 {len(gdf_restaurants)} 家餐厅 ---")

    # 加载最终的加权多边形
    gdf_final_polygons = gpd.read_file(FILE_FINAL_POLYGONS_WEIGHTED)

    # 仅保留用于连接的关键列 (和几何图形)
    cols_to_keep = [
        'zone', 'LocationID', 'Count_Lunch', 'Count_WkdyEve', 'Count_WkndEve',
        'weighted_score', 'Zs', 'p_sim', 'geometry'
    ]
    # 过滤掉 gdf_final_polygons 中可能不存在的列
    cols_to_keep_existing = [col for col in cols_to_keep if col in gdf_final_polygons.columns]
    gdf_final_polygons_slim = gdf_final_polygons[cols_to_keep_existing]

    # --- 核心：空间连接 (Spatial Join) ---
    # 将多边形的属性 (如 weighted_score) 附加到在多边形内的餐厅点上
    gdf_restaurants_tagged = gpd.sjoin(
        gdf_restaurants,
        gdf_final_polygons_slim,
        how='left',
        predicate='within' # Changed 'op' to 'predicate'
    )

    # 清理 sjoin 产生的 'index_right' 列
    if 'index_right' in gdf_restaurants_tagged.columns:
        gdf_restaurants_tagged = gdf_restaurants_tagged.drop(columns=['index_right'])

    gdf_restaurants_tagged.to_file(OUTPUT_FILE_C, driver="GeoJSON")
    print(f"\n--- 成功！脚本 C 已完成 ---")
    print(f"已将 {len(gdf_restaurants_tagged)} 家餐厅（包含已标记和未标记的）保存到 GeoJSON。")
    print(f"最终餐厅数据库已保存到: {OUTPUT_FILE_C}")

    print("\n--- 阶段二 (重新执行) 已全部完成 ---")
    print("您可以下载以下三个新文件：")
    print(f"1. {FILE_HOTSPOT_ARRIVALS_WEIGHTED}")
    print(f"2. {OUTPUT_FILE_B}")
    print(f"3. {OUTPUT_FILE_C}")

except FileNotFoundError as e:
    print(f"\n!!! 致命错误: 找不到文件 {e.filename} !!!")
    print("请确保 'restaurants_nyc_googlemaps.csv' 和 'final_hotspot_polygons_weighted.geojson' 存在。")
except Exception as e:
    print(f"\n!!! 脚本 C 失败: {e} !!!")


--- 开始执行脚本 B: 生成最终热点多边形 ---
--- 成功！脚本 B 已完成 ---
识别出 28 个最终融合的热点多边形。
结果已保存到: final_hotspot_polygons_weighted.geojson

--- 开始执行脚本 C: 生成餐厅数据库 ---
--- 成功加载 14330 家餐厅 ---

--- 成功！脚本 C 已完成 ---
已将 14330 家餐厅（包含已标记和未标记的）保存到 GeoJSON。
最终餐厅数据库已保存到: restaurants_with_hotspot_scores.geojson

--- 阶段二 (重新执行) 已全部完成 ---
您可以下载以下三个新文件：
1. hotspot_arrival_areas_weighted.geojson
2. final_hotspot_polygons_weighted.geojson
3. restaurants_with_hotspot_scores.geojson
